In [10]:
# Run this if the library is not installed
# pip install jsonlines

In [11]:
# Import libraries
import pandas as pd
import json
import io
import jsonlines
from IPython.display import display


In [14]:
from google.colab import files

uploaded = files.upload()

Saving users.json to users.json
Saving brands.json to brands.json
Saving receipts.json to receipts.json


In [40]:
# Load the raw JSON files
with open('/content/receipts.json', 'r', encoding='utf-8') as f:
    receipts = [json.loads(line) for line in f]

with open('/content/users.json', 'r', encoding='utf-8') as f:
    users = [json.loads(line) for line in f]

with open('/content/brands.json', 'r', encoding='utf-8') as f:
    brands = [json.loads(line) for line in f]


In [41]:
#Normalize into flat DataFrames
df_receipts = pd.json_normalize(receipts, sep='_')
df_users    = pd.json_normalize(users,    sep='_')
df_brands   = pd.json_normalize(brands,   sep='_')


In [46]:
# Dispaly dataframes
df_receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id_$oid,createDate_$date,dateScanned_$date,finishedDate_$date,modifyDate_$date,pointsAwardedDate_$date,purchaseDate_$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [45]:
# Dispaly dataframes
df_users.head()

,active,role,signUpSource,state,_id_$oid,createdDate_$date,lastLogin_$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12


In [47]:
# Dispaly dataframes
df_brands.head()

,barcode,category,categoryCode,name,topBrand,_id_$oid,cpg_$id_$oid,cpg_$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [48]:
# Convert millisecond‐epoch date fields to datetime
date_cols_receipts = [c for c in df_receipts.columns if c.endswith('_$date')]
for c in date_cols_receipts:
    clean_name = c.replace('_$date', '')
    df_receipts[clean_name] = pd.to_datetime(df_receipts[c], unit='ms')

date_cols_users = [c for c in df_users.columns if c.endswith('_$date')]
for c in date_cols_users:
    clean_name = c.replace('_$date', '')
    df_users[clean_name] = pd.to_datetime(df_users[c], unit='ms')


In [49]:
# EXPLORATORY DATA ANALYSIS
# 1. Missing‐value summary
print("=== Missing Values ===")
print("-- Receipts")
print(df_receipts.isna().sum()[lambda s: s>0].sort_values(), "\n")
print("-- Users")
print(df_users.isna().sum()[lambda s: s>0].sort_values(), "\n")
print("-- Brands")
print(df_brands.isna().sum()[lambda s: s>0].sort_values(), "\n")

# 2. Duplicate‐key checks (use the correct '_id_$oid' column)
print("=== Duplicate Primary Keys ===")
print("Receipts _id duplicates:", df_receipts['_id_$oid'].duplicated().sum())
print("Users    _id duplicates:", df_users   ['_id_$oid'].duplicated().sum())
print("Brands   _id duplicates:", df_brands  ['_id_$oid'].duplicated().sum(), "\n")


# 3. Referential‐integrity checks
print("=== Referential Integrity ===")
# 3a. Every receipts.userId should exist in users._id_$oid
bad_users = ~df_receipts['userId'].isin(df_users['_id_$oid'])
print("Receipts with userId NOT in users:", bad_users.sum())

# 3b. Expand receipts → items and check each item.barcode against brands.barcode
items = (
    df_receipts[['_id_$oid','rewardsReceiptItemList']]
      .explode('rewardsReceiptItemList')
)
# Split the dict in rewardsReceiptItemList into columns
items = pd.concat([
    items[['_id_$oid']],
    items['rewardsReceiptItemList'].apply(pd.Series)
], axis=1)
bad_brands = ~items['barcode'].isin(df_brands['barcode'])
print("Items with barcode NOT in brands:", bad_brands.sum(), "\n")

# 4. Data‐type consistency for key numeric fields
print("=== Data‐Type Consistency ===")
for col in ('pointsEarned','totalSpent'):
    coerced = pd.to_numeric(df_receipts[col], errors='coerce')
    print(f"{col}: coercion → nulls = {coerced.isna().sum()}")
print()

# 5. Logical date‐sequence checks
print("=== Logical Date Sequencing ===")
# e.g., purchaseDate should be on or before dateScanned
violations = (df_receipts['purchaseDate'] > df_receipts['dateScanned']).sum()
print("Receipts where purchaseDate > dateScanned:", violations, "\n")

# 6. Outliers & unexpected values
print("=== Outliers & Unexpected Values ===")
spent_num = pd.to_numeric(df_receipts['totalSpent'], errors='coerce')
print("Receipts with totalSpent ≤ 0:", (spent_num <= 0).sum())
threshold = df_receipts['purchasedItemCount'].quantile(0.99)
high_counts = df_receipts[df_receipts['purchasedItemCount'] > threshold]
print("PurchasedItemCount > 99th percentile:", sorted(high_counts['purchasedItemCount'].unique()), "\n")

# 7. Summary table of all issue counts
summary = {
    'missing_receipt_cols':     int((df_receipts.isna().sum()>0).sum()),
    'missing_user_cols':        int((df_users.isna().sum()>0).sum()),
    'missing_brand_cols':       int((df_brands.isna().sum()>0).sum()),
    'dup_receipt_ids':          int(df_receipts['_id_$oid'].duplicated().sum()),
    'dup_user_ids':             int(df_users   ['_id_$oid'].duplicated().sum()),
    'dup_brand_ids':            int(df_brands  ['_id_$oid'].duplicated().sum()),
    'broken_user_refs':         int(bad_users.sum()),
    'broken_brand_refs':        int(bad_brands.sum()),
    'pointsEarned_coercion_nulls': int(pd.to_numeric(df_receipts['pointsEarned'], errors='coerce').isna().sum()),
    'date_sequence_violations': int(violations),
    'nonpos_spend_count':       int((spent_num <= 0).sum())
}
print("=== Issue Summary ===")
display(pd.DataFrame.from_dict(summary, orient='index', columns=['count']))

=== Missing Values ===
-- Receipts
totalSpent                 435
rewardsReceiptItemList     440
purchaseDate               448
purchaseDate_$date         448
purchasedItemCount         484
pointsEarned               510
finishedDate_$date         551
finishedDate               551
bonusPointsEarnedReason    575
bonusPointsEarned          575
pointsAwardedDate_$date    582
pointsAwardedDate          582
dtype: int64 

-- Users
signUpSource       48
state              56
lastLogin_$date    62
lastLogin          62
dtype: int64 

-- Brands
category        155
brandCode       234
topBrand        612
categoryCode    650
dtype: int64 

=== Duplicate Primary Keys ===
Receipts _id duplicates: 0
Users    _id duplicates: 283
Brands   _id duplicates: 0 

=== Referential Integrity ===
Receipts with userId NOT in users: 148
Items with barcode NOT in brands: 7299 

=== Data‐Type Consistency ===
pointsEarned: coercion → nulls = 510
totalSpent: coercion → nulls = 435

=== Logical Date Sequencing ===


,count
missing_receipt_cols,12
missing_user_cols,4
missing_brand_cols,4
dup_receipt_ids,0
dup_user_ids,283
dup_brand_ids,0
broken_user_refs,148
broken_brand_refs,7299
pointsEarned_coercion_nulls,510
date_sequence_violations,13


## Summary of Exploratory Data Analysis

1. **435 receipts missing `totalSpent`**  
   These transactions lack a recorded spend amount, making revenue and average-basket analyses incomplete.

2. **283 duplicate user IDs**  
   Indicates multiple entries for the same user, which can inflate active-user counts and skew engagement metrics.

3. **148 receipts reference nonexistent users**  
   Those scans point to user IDs not in our user table, breaking the join and hiding actual user behavior.

4. **7,299 items reference unknown barcodes**  
   A large portion of purchased items can’t be matched to brands, undermining category or loyalty reporting.

5. **510 invalid `pointsEarned` values**  
   These entries contain non-numeric or malformed point amounts, preventing accurate loyalty rewards tallies.

6. **13 date-sequence violations** (`purchaseDate` > `dateScanned`)  
   A handful of receipts show a purchase occurring after the scan date, indicating clock or data entry errors.

7. **15 receipts with zero or negative spend**  
   These receipts report no or negative totals, which are clearly incorrect and distort average spend figures.
